# Crime Statistics

In [ ]:
# Setup Web Driver: Initialize the Chrome WebDriver.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

### Suburb List
suburb.xlsx is filtered from all_perth_310121.csv by removing duplicates in excel

In [ ]:
# Load the Excel file
file_path = 'Resources/suburb.xlsx'
df = pd.read_excel(file_path)

# Convert a specific column to a list
column_list = df['SUBURB'].tolist()

# List of suburbs to search
suburbs = column_list

In [ ]:
# Set up the web driver (make sure you have the correct path to your web driver)
driver = webdriver.Chrome()

# Define url
url = 'https://www.police.wa.gov.au/Crime/CrimeStatistics#/start'

# Open the website
driver.get(url)

### Crime Data
- The crime data selected is by calender year instead of financial year. This is to ensure the annual timeline starts from January to December rather than June to July the following year.

- The crime data is also filtered by each suburb.


In [ ]:
# Wait for the Calendar Year button to be clickable and click it
calendar_year_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-primary.btn-xs.btn-primary-crimestats[data-ng-click="selectCalendarYear()"]'))
)
calendar_year_button.click()

In [ ]:
# Click on Suburb
driver.find_element(By.CSS_SELECTOR, '.btn-locality-group[ng-model="location"][uib-btn-radio="\'Suburb\'"]').click()

# Scrape data from WA Police Website
The data from 2024 is removed as it is still incomplete and might impact the mean crime data across the years if required.

In [ ]:
# Initialize a new DataFrame to collect all data
all_data = pd.DataFrame()

In [ ]:
# Function to get crime statistics for a given suburb
def get_crime_statistics(suburb):
    global all_data
    
    try:   

        # Find the input field by its ID and input the suburb name
        input_field = driver.find_element(By.ID, 'searchLocality_value')
        input_field.clear()  # Clear any previous input
        input_field.send_keys(suburb)
        
        # Wait for the dropdown list to load
        time.sleep(0.5)
        
        # Find and select the correct item from the dropdown
        dropdown_item = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//div[@class='angucomplete-title ng-binding ng-scope' and contains(text(), '{suburb}')]"))
        )
        dropdown_item.click()  

        # Wait for the results to load
        time.sleep(0.5)

        # Check if the results loaded correctly
        if "No results found" in driver.page_source:
            print(f"No results found for suburb '{suburb}'. Skipping.")
            return all_data
        
        # Now use pandas to read the HTML source
        tables = pd.read_html(driver.page_source, attrs={'class': 'table table-striped table-condensed table-offences-stats'})

        # Check if any tables were found
        if tables:
            df = tables[0]
            # Filter the table
            df_filter = df.iloc[18, 1:10]
            # Modify to put years a column
            df_modify = pd.DataFrame(df_filter).transpose()
            # Insert the new column at position 0 (the first position)
            df_modify.insert(0, 'Suburbs', suburb)
            # Append to all_data DataFrame
            all_data = pd.concat([all_data, df_modify], ignore_index=True)
        else:
            print("Table not found.")

    except Exception as e:
        print(f"An error occurred for suburb {suburb}: {e}")
    return all_data

In [ ]:
# Loop through the list of suburbs and get the crime statistics
for suburb in suburbs:
    print(f"Getting data for {suburb}...")
    all_data = get_crime_statistics(suburb)

In [ ]:
all_data = all_data.rename(columns={
   'Suburbs':'Suburbs',
    '2014-15':2015,
    '2015-16':2016,
    '2016-17':2017,
    '2017-18':2018,
    '2018-19':2019,
    '2019-20':2020,
    '2020-21':2021,
    '2021-22':2022,
    '2022-23':2023
})

In [ ]:
# Make a copy of the scrapped data
all_data_copy = all_data
all_data_copy

In [ ]:
# Amend ['MANNING', 'STIRLING', 'TREEBY', 'VICTORIA PARK'] data
suburbs_to_update = {
    'MANNING':[453, 352, 309, 296, 255, 192, 337, 396, 503],
    'STIRLING':[571, 435, 446, 515, 507, 266, 275, 417, 396],
    'TREEBY':[19, 128, 116, 216, 174, 185, 164, 144],
    'VICTORIA PARK':[1815, 1681, 1581, 1296, 1279, 1215, 1493, 1367, 1687]
}

for suburb, new_data in suburbs_to_update.items():
    row_index = all_data_copy[all_data_copy['Suburbs'] == suburb].index
    if not row_index.empty:  # Check if the row is found
        all_data_copy.loc[row_index, all_data_copy.columns[1:len(new_data)+1]] = new_data

In [ ]:
# Add "O'CONNOR" data
# Define the new data as a dictionary
new_data = {
    'Suburbs': "O'CONNOR",
    2015:386,
    2016:413,
    2017:295,
    2018:417,
    2019:368,
    2020:298,
    2021:303,
    2022:327,
    2023:203
}

# Add the new row using loc
all_data_copy.loc[len(all_data_copy)] = new_data

In [ ]:
# Sort Suburbs alphabetically
df_sorted = all_data_copy.sort_values(by='Suburbs', ascending=True)

In [ ]:
# Reset the index and drop the old index
df_reset = df_sorted.reset_index(drop=True)

In [ ]:
# Remove redundant columns
df_cleaned = df_reset.iloc[:,0:10]

### Convert to .csv and saved
The suburb crime files are saved under Resources/suburb_crime

In [ ]:
# Save the combined DataFrame to CSV once all data is collected
filename = 'Resources/suburb_crime/combined_crime_statistics.csv'
df_cleaned.to_csv(filename, index=False)
print(f"Saved combined data to {filename}") 

In [ ]:
# Close the driver
driver.quit()